In [1]:
# + tags=["parameters"]
# declare a list tasks whose products you want to use as inputs
upstream = ['vectorizer_countVec']
random_seed = 42

In [2]:
# Parameters
random_seed = 42
upstream = {
    "vectorizer_countVec": {
        "nb": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\vectorizer_countVec.ipynb",
        "vectorizer": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\vectorizer_countVec.pkl",
    }
}
product = {
    "nb": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\topic_modeling_disaster_type_final.ipynb",
    "lda_model_earthquake": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_earthquake.pkl",
    "lda_model_fire": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_fire.pkl",
    "lda_model_flood": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_flood.pkl",
    "lda_model_hurricane": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_hurricane.pkl",
    "lda_topics_disaster_type": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_topics_disaster_type.csv",
}


importing all the functions defined in tools_rjg.py

In [3]:
from tools_rjg import *

In [4]:
import pandas as pd
import re, nltk, spacy, gensim
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gillrobe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
df_all = generate_disaster_type_dataframe(disaster_types = ('earthquake', 'fire', 'flood', 'hurricane')
                                          , dev_train_test= ('dev', 'train', 'test'))

In [8]:
df_all['tweet_text_cleaned'] = df_all['tweet_text'].apply(lambda x: tweet_preprocessing(x))
df_all['lemmatized'] = df_all['tweet_text_cleaned'].apply(lambda x: lemmatize_tweet_text(x, allowed_postags=('NOUN', 'ADJ', 'VERB', 'ADV')))

In [9]:
df_all.sample(100)

,tweet_id,tweet_text,class_label,disaster_type,tweet_text_cleaned,lemmatized
31627,783385314555531264,#VIDEO: Category 4 Hurricane #Matthew roars ac...,injured_or_dead_people,hurricane,"[video, category, hurricane, matthew, roars, a...",video category hurricane matthew roar least dead
6362,1111090467997999106,I am glad the state of Nebraska is doing this ...,not_humanitarian,flood,"[glad, state, nebraska, doing, this, this, sho...",glad state do national federal issue need prot...
254,910743978030489600,@TOPLatinAmerica @RealGDT @DunLaura @topupdati...,sympathy_and_support,earthquake,"[could, please, help, victims, earthquake, mex...",help victim earthquake thank sorry bother
1623,1023277928120037376,RT @LazarosKokkalis: There is no change runnin...,other_relevant_information,fire,"[there, change, running, same, policies, recon...",change run same policy reconstitution greece w...
3410,721755929553866753,CLICK HERE to support ECUADOR Earthquake Emerg...,rescue_volunteering_or_donation_effort,earthquake,"[click, here, support, ecuador, earthquake, em...",click here support ecuador earthquake emergenc...
...,...,...,...,...,...,...
5993,901735529196257280,RT @NBCNews: UPDATE: Hundreds of water rescues...,rescue_volunteering_or_donation_effort,hurricane,"[update, hundreds, water, rescues, underway, h...",update hundred water rescue underway official
497,1064901044633825280,I’d like to see Jerry Brown’s bank statements....,not_humanitarian,fire,"[like, jerry, brown, bank, statements, wonder,...",statement wonder much money receive fuck resid...
722,1031517066262728705,#KeralaFloods #KeralaFloodRelief #KeralaFloodD...,rescue_volunteering_or_donation_effort,flood,"[kerala, floods, kerala, flood, relief, kerala...",flood relief flood donation officer unload ent...
5069,1066072682964357127,Tomorrow I’ll be brewing the home brew version...,rescue_volunteering_or_donation_effort,fire,"[tomorrow, brewing, home, brew, version, resil...",tomorrow brew home brew version proud check so...


## LDA Model

In [10]:
params_earthquake = {'n_components' : 3,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

params_fire = {'n_components' : 3,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

params_flood = {'n_components' : 3,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

params_hurricane = {'n_components' : 3,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

lda_model_earthquake = LatentDirichletAllocation(**params_earthquake)
lda_model_fire = LatentDirichletAllocation(**params_fire)
lda_model_flood = LatentDirichletAllocation(**params_flood)
lda_model_hurricane = LatentDirichletAllocation(**params_hurricane)


In [11]:
with open("output\\vectorizer_countVec.pkl", "rb") as f:
    vectorizer = pickle.load(f)

In [12]:
earthquake_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='earthquake']['lemmatized']))
fire_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='fire']['lemmatized']))
flood_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='flood']['lemmatized']))
hurricane_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='hurricane']['lemmatized']))

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['donate'] not in stop_words.
  warnings.warn(


In [13]:
lda_model_earthquake.fit_transform(earthquake_text_vectorized)

array([[0.29154506, 0.03225082, 0.67620412],
       [0.86657155, 0.06671199, 0.06671647],
       [0.0305357 , 0.64421989, 0.32524441],
       ...,
       [0.46176972, 0.17212281, 0.36610747],
       [0.04169084, 0.7953818 , 0.16292737],
       [0.55833641, 0.05615612, 0.38550746]])

In [14]:
lda_model_fire.fit_transform(fire_text_vectorized)

array([[0.16667992, 0.66664368, 0.1666764 ],
       [0.03061903, 0.03175387, 0.9376271 ],
       [0.04917578, 0.90140837, 0.04941586],
       ...,
       [0.49029856, 0.32447723, 0.18522422],
       [0.93192557, 0.03466824, 0.0334062 ],
       [0.01261004, 0.16977242, 0.81761754]])

In [15]:
lda_model_flood.fit_transform(flood_text_vectorized)

array([[0.17840612, 0.76961967, 0.05197422],
       [0.05453876, 0.77655022, 0.16891101],
       [0.37543094, 0.54121059, 0.08335847],
       ...,
       [0.70572085, 0.05447348, 0.23980567],
       [0.08338713, 0.58432106, 0.33229182],
       [0.52783743, 0.19424844, 0.27791413]])

In [16]:
lda_model_hurricane.fit_transform(hurricane_text_vectorized)

array([[0.28121157, 0.03652859, 0.68225984],
       [0.02753788, 0.14398589, 0.82847624],
       [0.02679498, 0.946857  , 0.02634802],
       ...,
       [0.88331774, 0.05754459, 0.05913767],
       [0.79952667, 0.03187349, 0.16859984],
       [0.03463321, 0.42531918, 0.54004762]])

In [17]:
with open(product['lda_model_earthquake'], "wb") as f:
    pickle.dump(lda_model_earthquake, f)
with open(product['lda_model_fire'], "wb") as f:
    pickle.dump(lda_model_fire, f)
with open(product['lda_model_flood'], "wb") as f:
    pickle.dump(lda_model_flood, f)
with open(product['lda_model_hurricane'], "wb") as f:
    pickle.dump(lda_model_hurricane, f)

## Performance Stats

In [18]:
# Log Likelyhood: Higher the better
print("Log Likelihood Earthquake: ", lda_model_earthquake.score(earthquake_text_vectorized))
print("Log Likelihood Fire: ", lda_model_fire.score(fire_text_vectorized))
print("Log Likelihood Flood: ", lda_model_flood.score(flood_text_vectorized))
print("Log Likelihood Hurricane: ", lda_model_hurricane.score(hurricane_text_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity Earthquake: ", lda_model_earthquake.perplexity(earthquake_text_vectorized))
print("Perplexity Fire: ", lda_model_fire.perplexity(fire_text_vectorized))
print("Perplexity Flood: ", lda_model_flood.perplexity(flood_text_vectorized))
print("Perplexity Hurricane: ", lda_model_hurricane.perplexity(hurricane_text_vectorized))

Log Likelihood Earthquake:  -489957.40154870134


Log Likelihood Fire:  -849270.4203478852


Log Likelihood Flood:  -1017564.9037416613


Log Likelihood Hurricane:  -3202283.332471243


Perplexity Earthquake:  1545.0721822945434


Perplexity Fire:  1653.9649026362408


Perplexity Flood:  1798.3129541093047


Perplexity Hurricane:  1895.7263098863632


## Extract Topic Keywords

In [19]:
topics_earthquake = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_earthquake, n_words=100, dname='earthquake'))
topics_fire = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_fire, n_words=100, dname='fire'))
topics_flood = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_flood, n_words=100, dname='flood'))
topics_hurricane = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_hurricane, n_words=100, dname='hurricane'))

df_topic_keywords = pd.concat([topics_earthquake,topics_fire,topics_flood,topics_hurricane],axis=0)
    # df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    # df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords = df_topic_keywords.transpose()
df_topic_keywords.head()

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_f

,earthquake_Topic 0,earthquake_Topic 1,earthquake_Topic 2,fire_Topic 0,fire_Topic 1,fire_Topic 2,flood_Topic 0,flood_Topic 1,flood_Topic 2,hurricane_Topic 0,hurricane_Topic 1,hurricane_Topic 2
Word 0,prayer,help,pray,death,help,home,help,relief,flooding,damage,help,need
Word 1,safe,donate,damage,toll,victim,lose,need,donate,state,storm,relief,help
Word 2,injure,death,aftershock,death toll,donate,destroy,relief,victim,damage,evacuation,cyclone,food
Word 3,thought,victim,building,camp,camp,burn,rescue,fund,disaster,destroy,donate,pray
Word 4,affect,rescue,area,dead,relief,many,food,help,year,wind,affect,shelter


In [20]:
df_topic_keywords.to_csv(product['lda_topics_disaster_type'])